In [ ]:
import pandas as pd
import chardet

#Metodo que lee el tipo de encoding para evitar errores de lectura
def smart_read_csv(file_path) -> pd.DataFrame:
    with open(file_path,'rb') as f:
        res = chardet.detect(f.read(1000))

    return pd.read_csv(file_path, encoding=res['encoding'], encoding_errors='replace')

#Leemos los documentos
try:
    df_orders = smart_read_csv('../data/orders.csv')
    df_orders_details = smart_read_csv('../data/order_details.csv')
    df_pizzas = smart_read_csv('../data/pizzas.csv')
    df_pizza_types = smart_read_csv('../data/pizza_types.csv')

    print('Files load completed succcesfully')
except Exception as e:
    print(f"Error: {e}")

Files load completed succcesfully


In [ ]:
#Une la cabecera los pedidos con el detalle del pedido
df_orders_master = pd.merge(df_orders, df_orders_details, how="left", on="order_id")

#Une él codigo de pizza con el tipo de pizza que representa (sus características)
df_pizzas_master = pd.merge(df_pizzas, df_pizza_types, how= "left", on="pizza_type_id")

#Genera la sábana completa de pedidos, en la cual se añade información de las pizzas solicitadas
df_master = pd.merge(df_orders_master, df_pizzas_master, how="left", on="pizza_id")
